In [43]:
import json
import json
from typing import List, Union
import itertools

import numpy as np
from collections import defaultdict


In [44]:
def estimate_pass_at_k(
    num_samples: Union[int, List[int], np.ndarray],
    num_correct: Union[List[int], np.ndarray],
    k: int
) -> np.ndarray:
    """
    Estimates pass@k of each problem and returns them in an array.
    """

    def estimator(n: int, c: int, k: int) -> float:
        """
        Calculates 1 - comb(n - c, k) / comb(n, k).
        """
        if n - c < k:
            return 1.0
        return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))

    if isinstance(num_samples, int):
        num_samples_it = itertools.repeat(num_samples, len(num_correct))
    else:
        assert len(num_samples) == len(num_correct)
        num_samples_it = iter(num_samples)

    return np.array([estimator(int(n), int(c), k) for n, c in zip(num_samples_it, num_correct)])


In [45]:
def parse_rescue(text):
    # Split the text into sections using "id:" as a delimiter

    sections = text.split('\n')
    
    # Initialize a list to store the parsed data
    parsed_data = []
    
    # Iterate through each section (excluding the first empty section)
    for section in sections:
        # Initialize a dictionary to store the parsed information for this section
        section_data = {}

        current_data = section.split(':')

        
        # Extract the ID
        section_data['id'] = (int(current_data[0].strip())-1)//10
        
        # Extract the status 1
        section_data['status1'] = current_data[1].strip()
        
        if section_data['status1'] == "success":
            section_data['status1'] = True
        else:
            section_data['status1'] = False
        

        # if section_data['status1'] != "success":
        #     continue

        # Extract the status 1
        # section_data['status2'] = current_data[2].strip()

        # # Extract the time
        # section_data['time'] = current_data[3].strip()

        # # Extract the pattern
        # section_data['pattern'] = current_data[4].strip()

        # Append the parsed data for this section to the list
        parsed_data.append(section_data)
    
    return parsed_data

In [46]:
names = ["Text_DaVinci_Raw_Output","Text_DaVinci_Refined_Output","GPT3.5_Raw_Output","GPT3.5_Refined_Output"]
for name in names:
    print(name)
    outfilename = f'./ReScue_Results/{name}_full_result.txt'

    with open(outfilename, 'r') as f:
        result = f.read()
    rescue_data = parse_rescue(result)
    # print(rescue_data[0])


    results = defaultdict(list)

    for r in rescue_data:
        results[r['id']].append(r)
    

    # Calculate pass@k.
    total, correct = [], []
    for result in results.values():
            passed = [r["status1"] for r in result]
            # print(passed)
            total.append(len(passed))
            correct.append(sum(passed))
    total = np.array(total)
    correct = np.array(correct)

    ks = [1,5,10]
    pass_at_k = {f"pass@{k}": estimate_pass_at_k(total, correct, k).mean()
                        for k in ks if (total >= k).all()}
    print(pass_at_k)

Text_DaVinci_Raw_Output
{'pass@1': 0.005380577427821522, 'pass@5': 0.017721743115443903, 'pass@10': 0.027559055118110236}
Text_DaVinci_Refined_Output
{'pass@1': 0.0027559055118110236, 'pass@5': 0.01089967920676582, 'pass@10': 0.01706036745406824}
GPT3.5_Raw_Output
{'pass@1': 0.0027559055118110236, 'pass@5': 0.01137357830271216, 'pass@10': 0.01837270341207349}
GPT3.5_Refined_Output
{'pass@1': 0.0022309711286089243, 'pass@5': 0.009988334791484397, 'pass@10': 0.01706036745406824}
